In [11]:
import os
import pandas as pd
import networkx as nx
from collections import defaultdict, Counter
import random
import numpy as np
import json
from data_loader.data_loaders import DataLoader
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import ttest_rel, ttest_ind

In [12]:
metrics = ['loss', 'accuracy', 'precision', 'recall', 'roc_auc',
     'pr_auc', 'f1_score', 'mcc']

def get_comparison_bars(runs, metrics, title):
    results_file = lambda run: f'saved/log/DrugCombDB/{run}/test_results.csv'

    df1 = pd.read_csv(results_file(runs[0]['path']))
    df2 = pd.read_csv(results_file(runs[1]['path']))

    fig = go.Figure(data=[
        go.Bar(name=runs[0]['name'], x=metrics, y=df1.mean()[metrics], error_y=dict(type='data', array=df1.std()[metrics])),
        go.Bar(name=runs[1]['name'], x=metrics, y=df2.mean()[metrics], error_y=dict(type='data', array=df2.std()[metrics]))
    ])
    fig.update_layout(barmode='group', title=title)
    fig.show()
    
    for metric in metrics:
        t, p = ttest_rel(df1[metric], df2[metric])
        print(f'{metric}: t={t}, p={p}')
        t, p = ttest_ind(df1[metric], df2[metric])
        print(f'{metric}: t={t}, p={p}')

Comparison between using graph and going graphless

In [13]:
runs = {
    0: {
        'path': '1206_182941',
        'name': "Uses Graph"
    },
    1: {
        'path': '1206_183245',
        'name': "Graphless"
    }
}
get_comparison_bars(runs, metrics[1:], "Impact of PPI Graph on Performance")

accuracy: t=1.24341668028089, p=0.2816127803992202
accuracy: t=0.8807080277616022, p=0.40415696718186045
precision: t=1.0230120194113728, p=0.36413464797972084
precision: t=0.39724755141430934, p=0.7015729340128334
recall: t=1.391455061926757, p=0.23647738057240442
recall: t=1.1634840108445386, p=0.27815873885471976
roc_auc: t=0.6404623400809211, p=0.55671974268217
roc_auc: t=0.5502717207841324, p=0.5971578725799047
pr_auc: t=-0.08586460534278428, p=0.9357002700279704
pr_auc: t=-0.0718818974236095, p=0.944460483519214
f1_score: t=1.2513949413352006, p=0.27898123580930495
f1_score: t=0.835932195614633, p=0.4274530317149271
mcc: t=1.246397359024204, p=0.28062687670709824
mcc: t=0.8615882345723568, p=0.41399182380760946


Comparison between cleanup and no cleanup

In [14]:
runs = {
    0: {
        'path': '1206_182941',
        'name': "cleaned up"
    },
    1: {
        'path': '1206_185325',
        'name': "not cleaned up"
    }
}
get_comparison_bars(runs, metrics[1:], "Impact of Neighborhood Cleaning on Performance")

accuracy: t=1.3635821617222597, p=0.24439742945362225
accuracy: t=0.6276590629917481, p=0.5477204654439113
precision: t=1.6457697838247052, p=0.17515650038772265
precision: t=0.3814394061808027, p=0.7128084979899947
recall: t=0.48925536513911205, p=0.6502769271636519
recall: t=0.19923162489663127, p=0.8470521121505106
roc_auc: t=-0.5118070923753988, p=0.6357465598742743
roc_auc: t=-0.2347361957203888, p=0.8203100137406858
pr_auc: t=0.2606884132262771, p=0.8072033140437269
pr_auc: t=0.09821921452150109, p=0.924174581138018
f1_score: t=1.2088332824601542, p=0.2932949255165158
f1_score: t=0.41974937532191026, p=0.6857142453090821
mcc: t=1.4272066220313055, p=0.22669034205690064
mcc: t=0.6152182708070042, p=0.555502879812108


In [15]:
runs = {
    0: {
        'path': '1206_182941',
        'name': "Uses Graph"
    },
    1: {
        'path': '1207_004035',
        'name': "Graphless"
    }
}
get_comparison_bars(runs, metrics[1:], "Impact of PPI Graph on Performance (different parameters)")

accuracy: t=0.7271559950855514, p=0.5074053897777879
accuracy: t=0.4618264831608488, p=0.6565046228810371
precision: t=0.4192322616819418, p=0.6965814134196993
precision: t=0.1407511802638749, p=0.8915460197302164
recall: t=1.635698282511843, p=0.17724299270491442
recall: t=1.149317983265661, p=0.2836234836936246
roc_auc: t=0.0075817980045939995, p=0.9943137195938873
roc_auc: t=0.005741670193644198, p=0.9955594309632111
pr_auc: t=-0.7590427766789034, p=0.4900966148558366
pr_auc: t=-0.4660528429726561, p=0.6536040547328785
f1_score: t=1.099002606376702, p=0.3334702052999364
f1_score: t=0.6284997279315183, p=0.547196915669921
mcc: t=0.8424838641815953, p=0.4469441253158185
mcc: t=0.5109767008622084, p=0.6231667340964026
